<img height="40px" src="https://science-tokyo.box.com/shared/static/c6lh35sak4xkw9hbs2nqnyaptnubutej.png">

In [ ]:
# Preparation for auto-grading
unit = "DS3"
!pip install otter-grader 
!wget https://github.com/keisuke-yanagisawa/isct-efdsai/raw/refs/heads/main/supplementary_materials/{unit}/student.tar.gz -O {unit}.tar.gz
!tar -zxvf {unit}.tar.gz 
!rm {unit}.tar.gz
%cd student
import otter
grader = otter.Notebook()

# 基盤データサイエンス演習 第3回 決定木による分類とモデル評価 補習資料

# Supplemental Material: Excersices in Fundamentals of Data Science (3) - Classification and Model Evaluation Using Decision Trees

## 1 | 決定木を用いた分類

## 1 | Classification Using Decision Trees

#### 事前準備

#### Preliminaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Creating virtual data
np.random.seed(42)
n = 40
X_train = np.random.randn(n, 2)

noise = 1.6 * np.random.randn(n)                             # Noise is included in the results
y_train_bool = - X_train[:,0] + 2 * X_train[:,1] + noise > 0 # -x1 + 2 x2 > 0 is defined as positive
y_train = np.where(y_train_bool, 1, -1)                      # Convert so that True = 1, False = -1

In [ ]:
X_test = np.random.randn(20, 2)
y_test_bool = - X_test[:,0] + 2 * X_test[:,1] > 0
y_test = np.where(y_test_bool, 1, -1)

In [ ]:
plt.scatter(X_train[y_train==1,0], X_train[y_train==1,1], marker="o", label="train - Positive")
plt.scatter(X_train[y_train==-1,0], X_train[y_train==-1,1], marker="o", label="train - Negative")
plt.scatter(X_test[y_test==1,0], X_test[y_test==1,1], marker="x", label="test  - Positive")
plt.scatter(X_test[y_test==-1,0], X_test[y_test==-1,1], marker="x", label="test  - Negative")
plt.legend()

#### 練習問題 1-1 | 決定木の構築

#### Exercise 1-1 | Constructing a Decision Tree

最大深さ 5 の決定木を構築せよ。ただし、決定木の最大深さは `max_depth` で指定できる。

Construct a decision tree with a maximum depth of 5. The maximum depth of the decision tree can be specified with `max_depth`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
q1_dt = ...
q1_dt.fit(X_train, y_train)

In [ ]:
grader.check("Exercise 1-1")

#### 練習問題 1-2 | 予測の実施

#### Exercise 1-2 | Making Predictions

`X_test = [[1,4], [-3,1]]` を予測し、その予測結果を `q2_y_pred` に代入せよ。

Predict the value for `X_test = [[1,4], [-3,1]]` and assign the prediction result to `q2_y_pred`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
q2_dt = DecisionTreeClassifier()
q2_dt.fit(X_train, y_train)

In [ ]:
q2_X_test = [[1,4], [-3,1]]
q2_y_pred = ...

In [ ]:
grader.check("Exercise 1-2")

#### 練習問題 1-3 | 予測結果の評価

#### Exercise 1-3 | Evaluating Prediction Results


ある予測モデルを使って `X_pred` を予測した時の結果 `y_pred` は以下の通りであった。
```
q3_y_pred = [1,0,0,1,1,1,0,1,1,1]
```
一方、そのデータの真のラベル `y_true` は以下の通りであった。
```
q3_y_true = [0,0,0,1,0,1,0,1,1,1]
```
この時の予測正解率 `q3_accuracy` を求めよ。（計算結果は0.8となるはずである。）

When using a prediction model, the results of predicting `X_pred` yielded the following `y_pred`:
```
q3_y_pred = [1,0,0,1,1,1,0,1,1,1]
```
On the other hand, the true labels for that data `y_true` were as follows:
```
q3_y_true = [0,0,0,1,0,1,0,1,1,1]
```
Calculate the prediction accuracy `q3_accuracy` at this time. (The calculation result should be 0.8.)

In [ ]:
from sklearn.metrics import accuracy_score

q3_y_pred = [1,0,0,1,1,1,0,1,1,1]
q3_y_true = [0,0,0,1,0,1,0,1,1,1]

q3_accuracy = ...

In [ ]:
grader.check("Exercise 1-3")

#### 練習問題 1-4 | 予測モデルの構築、新規データの予測と性能評価

#### Exercise 1-4 | Building a Prediction Model, Making Predictions on New Data, and Performance Evaluation

以下のコードを完成させ、予測精度 `q4_accuracy` を算出せよ。

Complete the following code to calculate the prediction accuracy `q4_accuracy`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Construct prediction model
q4_dt = DecisionTreeClassifier()
...

# Predict test data
q4_y_pred = ...

# Evaluate prediction accuracy
q4_accuracy = accuracy_score(y_test, q4_y_pred)


In [ ]:
grader.check("Exercise 1-4")

## 2 | 交差検証法を用いた汎化性能評価

## 2 | Generalization Performance Evaluation Using Cross-Validation

#### 事前準備

#### Preparation

In [ ]:
from sklearn import datasets

# breast cancer dataset: predict whether the cancer is malignant or benign
q2_X, q2_y = datasets.load_breast_cancer(return_X_y=True)

#### 練習問題 2-1 | 単一の機械学習手法の汎化性能評価


#### Exercise 2-1 | Evaluating the Generalization Performance of a Single Machine Learning Method


最大深さが5になるような決定木を構築するオブジェクト `q21_dt_d5` を定義し、`cross_val_score()` 関数を用いて、与えられた分類タスクに対する汎化性能 generalization performance を `q21_gen_perf_score` 変数に格納せよ。ここで言う「汎化性能」は、予測正解率のことを指すものとする。また、ここでの交差検証は5分割で実施せよ。

Define an object `q21_dt_d5` to construct a decision tree with a maximum depth of 5, and use the `cross_val_score()` function to store the generalization performance for the given classification task in the variable `q21_gen_perf_score`. Here, "generalization performance" refers to prediction accuracy. Also, perform 5-fold cross-validation.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
random_state = 59

q21_dt_d5 = DecisionTreeClassifier(
    max_depth= ...
    random_state=random_state) 

q21_gen_perf_score = ...
q21_gen_perf_score = q21_gen_perf_score.mean()


In [ ]:
grader.check("Exercise 2-1")

#### 練習問題 2-2 | 他者のモデルとの汎化性能比較

#### Exercise 2-2 | Comparing Generalization Performance with Other Models

自分とは異なる研究者が `q22_dt_other = DecisionTreeClassifier(min_impurity_decrease=0.1)` が最良の予測モデルを作成すると過去に提案していたとする。5分割の交差検証法を用いて、他者の手法の汎化性能を計算、 `q22_other_gen_perf_score` に格納せよ。
なお、他者の手法よりも練習問題2-1で提案した手法の方が汎化性能が高くなるはずである。

Assume that a different researcher previously proposed that `q22_dt_other = DecisionTreeClassifier(min_impurity_decrease=0.1)` would create the best prediction model. Use 5-fold cross-validation to calculate the generalization performance of the other person's method and store it in `q22_other_gen_perf_score`.
Note that the method you proposed in Exercise 2-1 should have higher generalization performance than the other person's method.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

q22_dt = ...

q22_gen_perf_score = ...
q22_gen_perf_score = q22_gen_perf_score.mean()

In [ ]:
grader.check("Exercise 2-2")

## 3 | モデル選択・グリッドサーチ


## 3 | Model Selection and Grid Search


#### 事前準備

#### Preparation

2 で作成した `q2_X`, `q2_y` をそのまま用いる。

Use `q2_X` and `q2_y` created in section 2 as they are.

In [ ]:
from sklearn.model_selection import train_test_split
q3_X_train, q3_X_test, q3_y_train, _ = train_test_split(q2_X, q2_y, test_size=0.2, random_state=42)

#### 練習問題 3-1 | 予測の実施

#### Exercise 3-1 | Making Predictions

訓練データ `q3_X_train`, `q3_y_train` を用いて予測モデルを構築し、テストデータに対する予測結果 `q31_test_predicted` を求めよ。ただし、予測モデルの構築時には `q31_dt = DecisionTreeClassifier()` を用いよ。

Construct a prediction model using the training data `q3_X_train`, `q3_y_train`, and determine the prediction results `q31_test_predicted` for the test data. When constructing the prediction model, use `q31_dt = DecisionTreeClassifier()`.

In [ ]:
q31_dt = DecisionTreeClassifier(random_state=random_state)
...
q31_test_predicted = ...

In [ ]:
grader.check("Exercise 3-1")

#### 練習問題 3-2 | グリッドサーチによるハイパーパラメータチューニング

#### Exercise 3-2 | Hyperparameter Tuning Using Grid Search

`GridSearchCV()` を用いて、 `max_depth` を1から10まで変化させて、最も良い汎化性能を示す `max_depth` の値を決定せよ。グリッドサーチを行う際には、5分割の交差検証を行うこと。

Using `GridSearchCV()`, vary `max_depth` from 1 to 10, and determine the value of `max_depth` that demonstrates the best generalization performance. Perform a 5-fold cross-validation during the grid search.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

q32_dt = DecisionTreeClassifier(random_state=random_state)
q32_param_grid = {
    ...
}

q32_gs = GridSearchCV(q32_dt, q32_param_grid, cv=5)
q32_gs.fit(q3_X_train, q3_y_train)

q32_best_params = ...

In [ ]:
grader.check("Exercise 3-1")